In [1]:
#Import all importtant libraries
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma  # Chroma vector store
from langchain.vectorstores import FAISS  # FAISS vector store for fast local search
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import CTransformers
#from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
#import faiss
#from transformers import pipeline 

In [2]:
%pwd

'd:\\Final _proj\\medical chatbot\\Chatbot original\\research'

In [3]:
import os 
os.chdir("..")

In [4]:
def load_pdf(data):      #Load the pdf from data floder
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)  #Load only pdf file
    documents = loader.load() 
    return documents

documents = load_pdf(data="data/")
len(documents)

428

In [6]:
def text_split(extracted_data): 
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)  # Larger chunks for fewer docs
    text_chunks = text_splitter.split_documents(extracted_data) 
    return text_chunks 

text_chunk = text_split(documents)      #Split the data imto small chunks
print("Length of Text chunk:", len(text_chunk))

Length of Text chunk: 1879


In [6]:
from transformers import pipeline

# Initialize the Hugging Face model for text-to-text generation (consider using a better model if available)
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")

# Define the prompt template for generating questions based on context and answer
QUESTION_PROMPT_TEMPLATE = (
    "Based on the following context and answer, generate a clear and meaningful question:\n\n"
    "Context: {content}\n"
    "Answer: {answer}\n\n"
    "Question:"
)

# Define the prompt template for generating answers based on context and question
ANSWER_PROMPT_TEMPLATE = (
    "Using the following context, provide a concise, informative, and understandable answer to the question:\n\n"
    "Question: {question}\n"
    "Context: {content}\n"
    "Answer:"
)

# List to store the generated question-answer pairs
qa_pairs = []

# Iterate through the documents and generate questions and answers
for doc in documents[:5]:
    content = doc.page_content

    # Extracting a meaningful answer from content
    if "is" in content:
        answer = content.split("is")[-1].strip().rstrip(".")
    else:
        answer = "Unknown"

    # Create prompt for generating a question based on the context and answer
    question_prompt = QUESTION_PROMPT_TEMPLATE.format(content=content, answer=answer)
    
    # Generate question using the Hugging Face model (Flan-T5)
    question_response = qa_pipeline(question_prompt, max_length=100, num_return_sequences=1)[0]['generated_text']
    
    # Clean up response to ensure it is a valid question
    question = question_response.strip()
    
    # Create prompt for generating an answer based on the context and generated question
    answer_prompt = ANSWER_PROMPT_TEMPLATE.format(question=question, content=content)
    
    # Generate answer using the Hugging Face model (Flan-T5)
    # Set max_length to allow longer answers and min_length to enforce minimum length
    answer_response = qa_pipeline(answer_prompt, max_length=50, min_length=10, num_return_sequences=1)[0]['generated_text']
    
    # Clean up response to ensure it is a valid answer
    generated_answer = answer_response.strip()

    # Append the results to qa_pairs
    qa_pairs.append({"question": question, "answer": generated_answer})

# Print the final generated question-answer pairs
print("Generated Questions and Answers:")
for qa in qa_pairs:
    print(f"Question: {qa['question']}")
    print(f"Answer: {qa['answer']}")
    print()

Token indices sequence length is longer than the specified maximum sequence length for this model (669 > 512). Running this sequence through the model will result in indexing errors


Generated Questions and Answers:
Question: What is diabetes?
Answer: a condition that happens when your blood sugar (glucose) is too high

Question: What are the three most common forms of diabetes?
Answer: Type 2 diabetes is the most common form, representing 90% to 95% of all diabetes cases

Question: What is the most common type of diabetes?
Answer: Type 1 diabetes. These symptoms are usually more intense in Type 1 diabetes than Type 2 diabetes.

Question: What is the most common cause of diabetes?
Answer: Insulin resistance. Type 2 diabetes mainly results from insulin resistance. Several factors and conditions contribute to varying degrees of insulin resistance, including obesity, lack of physical activity, diet, hormonal imbalances, genetics and certain medications.

Question: What is the most common type of cardiovascular disease?
Answer: Coronary artery disease. br>



In [13]:
import random 
random.sample(text_chunk , k=10)

[Document(metadata={'source': 'data\\mental_health.pdf', 'page': 15}, page_content='pleasant activities\n If  a task seems too difficult, try breaking it  \n into a series of  small steps\n Above all reward yourself  for your efforts\n Avoid discussions of  bad feelings. Solving  \n problems is more helpful'),
 Document(metadata={'source': 'data\\Stroke disease pdf.pdf', 'page': 11}, page_content='The most common tests that happen when a healthcare provider suspects a stroke include: \n• Computerized tomography (CT) scan. \n• Lab blood tests (looking for signs of infections or heart damage, checking clotting ability and'),
 Document(metadata={'source': 'data\\mental_health.pdf', 'page': 15}, page_content='14\n Coping with depression\n Avoid sitting or lying about doing nothing\n Identify things you used to do regularly and \n  things which you used to enjoy\n Plan to gradually increase the routine of   \n pleasant activities'),
 Document(metadata={'source': 'data\\Stroke disease pdf.pd

In [7]:
from transformers import pipeline 
from tqdm import tqdm
import random
# Initialize the Hugging Face model for text-to-text generation (consider using a better model if available)
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")

# Define the prompt template for generating questions based on context and answer
QUESTION_PROMPT_TEMPLATE = (
    "Based on the following context and answer, generate a clear and meaningful question:\n\n"
    "Context: {content}\n"
    "Answer: {answer}\n\n"
    "Question:"
)

# Define the prompt template for generating answers based on context and question
ANSWER_PROMPT_TEMPLATE = (
    "Using the following context, provide a concise, informative, and understandable answer to the question:\n\n"
    "Question: {question}\n"
    "Context: {content}\n"
    "Answer:"
)

# List to store the generated question-answer pairs
qa_pairs = []

# Iterate through the documents and generate questions and answers
for doc in tqdm(random.sample(text_chunk , k=200)):
    content = doc.page_content

    # Extracting a meaningful answer from content
    if "is" in content:
        answer = content.split("is")[-1].strip().rstrip(".")
    else:
        answer = "Unknown"

    # Create prompt for generating a question based on the context and answer
    question_prompt = QUESTION_PROMPT_TEMPLATE.format(content=content, answer=answer)
    
    # Generate question using the Hugging Face model (Flan-T5)
    question_response = qa_pipeline(question_prompt, max_length=100, num_return_sequences=1)[0]['generated_text']
    
    # Clean up response to ensure it is a valid question
    question = question_response.strip()
    
    # Create prompt for generating an answer based on the context and generated question
    answer_prompt = ANSWER_PROMPT_TEMPLATE.format(question=question, content=content)
    
    # Generate answer using the Hugging Face model (Flan-T5)
    # Set max_length to allow longer answers and min_length to enforce minimum length
    answer_response = qa_pipeline(answer_prompt, max_length=50, min_length=10, num_return_sequences=1)[0]['generated_text']
    
    # Clean up response to ensure it is a valid answer
    generated_answer = answer_response.strip()

    # Append the results to qa_pairs
    qa_pairs.append({"context":content,"question": question, "answer": generated_answer  })

len(qa_pairs)

100%|██████████| 200/200 [16:20<00:00,  4.90s/it]


200

In [8]:
import pandas as pd
fine_tunning_data= pd.DataFrame(qa_pairs)

In [9]:
fine_tunning_data.to_json("fineTuneData.json") # saving data for fine tune in json format

In [10]:
# loading the data in json format 
pd.read_json('fineTuneData.json')

,context,question,answer
0,well as its association with subsequent suicid...,What is the main idea of the passage?,monitoring of population rates of inten- tiona...
1,duals. Journal of the American Geriatric Socie...,What is the meaning of symptom-check-list scor...,testing of multiple hypotheses. Social Science...
2,"Health, 86(5):674–677.\nSyme SL (2003). Psycho...",What is the population of the study that Syme ...,86(5):674–677
3,should be screened for G-6-PD deficiency befor...,What is the best time to give primaquine?,During pregnancy : Women who are pregnant should
4,of this publication to duplicate and distribu...,What is the purpose of the passage?,to duplicate and distribute as many copies as ...
...,...,...,...
195,5) Does the entire chain of command understan...,What is the most likely outcome of the counter...,enforcing the countermeasures
196,"the spleen \n• Decline in hemoglobin, hematoc...",If a large number of red blood cells are destr...,hemoglobinuria and symptoms if hemolysis severe
197,those patients with low-grade infections of lo...,What is the most common cause of malaria trans...,malariae infection may produce a unique immune...
198,Estimation and interpretation of blood glucose...,What is the only test for diagnosis of diabetes?,Estimation and interpretation of blood glucose...


# Fine-tunning

In [4]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset, Dataset

# Load your dataset
data_path = "fineTuneData.json"  # Path to your JSON file
dataset = Dataset.from_json(data_path)

# Tokenizer and Model
model_name = "google/flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
def preprocess_function(examples):
    # Ensure the inputs are strings
    inputs = [f"Question: {str(question)}" for question in examples['question']]
    targets = [str(answer) for answer in examples['answer']]
    
    # Tokenize the inputs and targets
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids
    
    # Add labels to the model inputs
    model_inputs["labels"] = labels
    return model_inputs

# Apply preprocessing to the dataset
processed_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names)
processed_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1
})

In [8]:
from datasets import Dataset

# Duplicate samples
duplicated_data = processed_dataset.to_pandas().sample(frac=10, replace=True)
processed_dataset = Dataset.from_pandas(duplicated_data)

# Perform train-test split
train_test_split = processed_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']


In [9]:
training_args = TrainingArguments(
    output_dir="./flan-t5-finetuned-new",
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=100,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    learning_rate=5e-5,  # Lower learning rate
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    fp16=True,  # Enable mixed precision for better performance
    push_to_hub=False,
)



# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

# Fine-Tune Model
trainer.train()

# Save the model locally
model.save_pretrained("./flan-t5-finetuned-new")
tokenizer.save_pretrained("./flan-t5-finetuned-new")


C:\Users\rohit\AppData\Local\Temp\ipykernel_15356\4194048252.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


  0%|          | 0/6 [00:00<?, ?it/s]

: 